InsightFace -> utilise ArcFace

In [1]:
"""from arcface import ArcFace

model = ArcFace(pretrained=True)
embedding = model.forward(image_tensor)"""


'from arcface import ArcFace\n\nmodel = ArcFace(pretrained=True)\nembedding = model.forward(image_tensor)'

In [2]:
!pip install torchvision
!pip install onnx==1.16.1
!pip install insightface==0.7.3


In [3]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from insightface.app import FaceAnalysis
from insightface.model_zoo import get_model
import cv2

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Chemins vers les datasets
dataset_path = "../../../ChimpRec-Dataset/Chimpanzee_recognition_dataset"

# Transformations d'image (redimensionnement, normalisation, etc.)
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Taille standard pour ArcFace
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalisation pour -1 à 1
])

# Chargement des datasets
train_dataset = datasets.ImageFolder(os.path.join(dataset_path, "train"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(dataset_path, "val"), transform=transform)

# Création des DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Mapping des classes
class_to_idx = train_dataset.class_to_idx
print("Classes :", class_to_idx)


Classes : {'FANLE': 0, 'FANWA': 1, 'FLANLE': 2, 'JEJE': 3, 'JOYA': 4, 'PELEY': 5}


In [4]:
from insightface.model_zoo import model_store

print("Modèles disponibles :", model_store)

ModuleNotFoundError: No module named 'insightface'

In [ ]:
arcface_model = get_model('arcface_r100_v1') 
arcface_model.prepare(ctx_id=0)


AttributeError: 'NoneType' object has no attribute 'prepare'

In [ ]:
# Ajouter une nouvelle tête de classification
class ArcFaceFineTuned(nn.Module):
    def __init__(self, arcface_model, num_classes):
        super(ArcFaceFineTuned, self).__init__()
        self.arcface = arcface_model
        self.fc = nn.Linear(512, num_classes)  # 512 est la dimension des embeddings ArcFace

    def forward(self, x):
        embeddings = self.arcface(x)  # Génère des embeddings
        logits = self.fc(embeddings)  # Logits pour la classification
        return embeddings, logits

model = ArcFaceFineTuned(arcface_model, num_classes).to(device)

# Définir la perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Fonction d'entraînement
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    epoch_loss, correct, total = 0.0, 0, 0
    for imgs, labels in tqdm(loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        _, logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, preds = torch.max(logits, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    accuracy = correct / total
    return epoch_loss / len(loader), accuracy

# Fonction de validation
def validate_epoch(model, loader, criterion, device):
    model.eval()
    epoch_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            _, logits = model(imgs)
            loss = criterion(logits, labels)
            epoch_loss += loss.item()
            _, preds = torch.max(logits, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return epoch_loss / len(loader), accuracy

# Entraînement du modèle
num_epochs = 10
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")

# Sauvegarde du modèle
torch.save(model.state_dict(), "fine_tuned_arcface_chimpanzees.pth")
print("Modèle fine-tuné sauvegardé.")


Classes : {'FANLE': 0, 'FANWA': 1, 'FLANLE': 2, 'JEJE': 3, 'JOYA': 4, 'PELEY': 5}
